# Day 1 실습 1: 데이터 전처리 & 검증

이 노트북에서는 파인튜닝에 사용할 한국어 RAG 데이터를 전처리하고, RAFT 템플릿에 맞게 검증까지 진행합니다.

### 목차
1. 데이터 로드
2. 스키마 정의 및 검증
3. 전처리 규칙 적용
4. RAFT 템플릿 변환
5. 통계 확인
6. 저장 및 Google Drive 백업

## 1. 데이터 로드
- Hugging Face의 `neural-bridge/rag-dataset-12000`를 사용합니다.
- 이미 context/question/answer 구조를 포함하고 있어 RAG 실험에 적합합니다.

In [ ]:
import json
import random
from typing import Dict, Any

import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset

print("✅ 라이브러리 로드 완료")

DATASET_NAME = "neural-bridge/rag-dataset-12000"
print(f"📥 데이터 로드: {DATASET_NAME}")

raw_dataset = load_dataset(DATASET_NAME, split="train")
print(f"   - 샘플 수: {len(raw_dataset):,}")
print(f"   - 필드: {raw_dataset.column_names}")

## 2. 스키마 정의 및 검증
Instruction/Input/Output 형식으로 정리해 파인튜닝에 쓰기 좋게 만듭니다.

In [ ]:
def normalize_record(record: Dict[str, Any]) -> Dict[str, Any]:
    """RAG 레코드를 Instruction/Input/Output 형태로 변환."""
    instruction = "다음 질문에 답변을 작성하세요."
    question = str(record.get("question", ""))
    context = str(record.get("positive_ctx", record.get("context", "")))
    answer = str(record.get("answer", record.get("positive_answer", "")))

    return {
        "instruction": instruction,
        "input": f"질문: {question}문서: {context}",
        "output": answer,
    }

normalized_samples = [normalize_record(raw_dataset[i]) for i in range(3)]
normalized_samples[0]

## 3. 전처리 규칙 적용
- 공백 정리, 중복 제거 등 기본 정리를 적용합니다.

In [ ]:
def clean_text(text: str) -> str:
    text = text.replace('​', ' ').replace(' ', ' ')
    text = ' '.join(text.split())
    return text.strip()

processed_records = []
seen_pairs = set()

for record in raw_dataset:
    normalized = normalize_record(record)
    normalized = {k: clean_text(v) for k, v in normalized.items()}

    pair = (normalized["instruction"], normalized["input"])
    if pair in seen_pairs:
        continue
    seen_pairs.add(pair)

    processed_records.append(normalized)

print("✅ 전처리 완료")
print(f"   - 중복 제거 후 샘플 수: {len(processed_records):,}")
processed_records[0]

## 4. RAFT 템플릿 변환
- 정답 문서(positive)와 distractor 문서를 포함하는 텍스트로 변환합니다.

In [ ]:
from tqdm.auto import tqdm

all_contexts = [normalize_record(item)["input"] for item in raw_dataset]

raft_records = []
for record in tqdm(processed_records, total=len(processed_records)):
    question_line = record["input"]
    answer = record["output"]
    context = question_line

    # distractor 선택 (간단히 랜덤 선택)
    distractors = []
    while len(distractors) < 2:
        candidate = random.choice(all_contexts)
        if candidate != context and candidate not in distractors:
            distractors.append(candidate)

    text = " ".join([
        question_line,
        "오답 문서: " + " || ".join(distractors),
        "위 자료를 참고해 자연스러운 답변을 작성하세요.",
    ])

    raft_records.append({
        "text": text,
        "question": record["input"],
        "answer": answer,
        "distractors": distractors,
    })

print("✅ RAFT 템플릿 변환 완료")
print(f"   - RAFT 샘플 수: {len(raft_records):,}")

## 5. 통계 확인
전처리 결과가 기대한 분포인지 간단히 살펴봅니다.

In [ ]:
text_lengths = [len(item["text"]) for item in raft_records]
print("📊 전처리 결과 요약")
print(f"  • 평균 텍스트 길이: {np.mean(text_lengths):.1f}")
print(f"  • 중앙값 텍스트 길이: {np.median(text_lengths):.1f}")
print(f"  • 최대 텍스트 길이: {max(text_lengths):,}")

## 6. 저장 및 Google Drive 백업

In [ ]:
raft_dataset = Dataset.from_list(raft_records)
SAVE_DIR = "processed_data"
raft_dataset.save_to_disk(SAVE_DIR)

stats = {
    "total_samples": len(raft_records),
    "avg_text_length": float(np.mean(text_lengths)),
}

with open("processed_data/metadata.json", "w", encoding="utf-8") as f:
    json.dump(stats, f, ensure_ascii=False, indent=2)

print("💾 저장 완료: processed_data/ 디렉터리")

In [ ]:
from huggingface_hub import login
from datasets import load_from_disk
from datasets import DatasetDict
ds = load_from_disk("processed_data")   # Dataset 또는 DatasetDict
print(ds)

login(token="")  # HF Access Token 입력

tmp = ds.train_test_split(test_size=0.1, seed=42)
ds = DatasetDict({
    "train": tmp["train"],
    "validation": tmp["test"]
})

repo_id = "ryanu/raft-processed"   # 원하는 repo 이름
ds.push_to_hub(repo_id, private=True)      # 공개하려면 private=False


In [ ]:
# GOOGLE_DRIVE_DIR = "/content/drive/MyDrive/exaone_day1/processed_data"
# try:
#     from google.colab import drive  # type: ignore
#     import shutil
#     drive.mount('/content/drive')
#     shutil.copytree("processed_data", GOOGLE_DRIVE_DIR, dirs_exist_ok=True)
#     print(f"✅ Google Drive에 백업 완료: {GOOGLE_DRIVE_DIR}")
# except ModuleNotFoundError:
#     print("⚠️ Colab 환경이 아닙니다. 필요하면 직접 경로를 수정해 저장하세요.")
# except NotImplementedError:
#     print("⚠️ 이 환경에서는 drive.mount()를 사용할 수 없습니다. Colab에서 실행해주세요.")

## 7. 다음 단계
- **00.03-raft-preprocessing.ipynb** 로 이동해 RAFT 템플릿 예시를 더 살펴보세요.
- **main-practice/02**에서는 저장된 `processed_data/`를 불러와 품질 검증을 진행합니다.